# Numerical Techniques 3: Space discretization



## Preparations: load numpy and matplotlib libraries

In [ ]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(phi,frames=None,labels=None,ylim=None):
    
    # create figure with axes
    fig,ax=plt.subplots()
    
    if len(phi.shape)==2:
        phi=phi.reshape([phi.shape[0],phi.shape[1],1])
    
    # number of experiments
    nExp=phi.shape[2]

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=phi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    
    # set y limits
    if ylim is None:
        ymin=np.min(phi[0,:,:])
        ymax=np.max(phi[0,:,:])
        ylim=[1.2*ymin-0.2*ymax,1.2*ymax-0.2*ymin]
    
    # labels and legend   
    showlegend=True
    if labels is None:
        showlegend=False
        labels=['exp %i'%jExp for jExp in range(nExp)]
        
    # create lines for numerical solution
    ll_n=[ax.plot(x,phi[frames[0],:,jExp],label=labels[jExp])[0] for jExp in range(nExp)]
    
    # set ylim
    ax.set_ylim(ylim)
    
    # add legend
    if showlegend:
        plt.legend()
    
    # add title
    tt=plt.title('')
    

    def animate(iframe):
        for jExp in range(nExp):
            ll_n[jExp].set_ydata(phi[iframe,:,jExp])
        tt.set_text('timestep %i/%i'%(iframe,nt))

    anim=animation.FuncAnimation(fig, animate, frames=frames,cache_frame_data=False,blit=True)
    display(anim)

## Phase speed and group speed

Phase speed and group speed can be illustrated by applying centered space differences to the advection equation. Time integration is done exactly. Letting

$$\phi_j(t)=\sum_k \hat\phi_k e^{-i(\omega t-kj\Delta x)}$$

and substituting in the advection equation

$$\frac{\partial \psi}{\partial t}+c\frac{\partial \psi}{\partial x}=0$$

leads to 

$$\omega \phi_k-\frac{c\sin k\Delta x}{\Delta x}\phi_k=0$$

so the dispersion relation is

$$\omega=c\frac{\sin k\Delta x}{\Delta x}$$

To visualize the difference, we animate $\phi(t)$ for an initial condition which is the sum of two waves:

$$\phi=e^{i(\omega_1 t-k_1j\Delta x)}+e^{i(\omega_2 t-k_2j\Delta x)}$$

By choosing $k_1$ and $k_2$ close together and high enough, the group speed will be negative, and the phase speed will be positive.


In [ ]:
# parameters
nt=100
dt=2*np.pi/nt
nx=128
dx=2*np.pi/nx
c=dx/dt

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# wavenumbers (-n/2:n/2)
k1=nx//8-4
k2=nx//8-5


# dispersion relation
omega1=c*np.sin(4*k1*dx)/dx
omega2=c*np.sin(4*k2*dx)/dx

# allocate numerical solution, initialize with zeros
phi=np.zeros((nt,nx))

# set initial state: sum of two harmonics
for jt in range(nt):
    phi[jt,:]=np.real(np.exp(1j*(omega1*jt*dt-k1*x))+np.exp(1j*(omega2*jt*dt-k2*x)))

# create animation
create_animation(phi=phi)

## Response to spike experiment

In [ ]:
# parameters
dt=1.0
nt=40
nx=128
dx=2*np.pi/nx
c=dx/dt

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# wavenumbers: conventional order is (0:n/2-1,-n/2:-1)
k=(np.arange(nx)+nx//2)%nx-nx//2

# dispersion relation
nExp=5
omega=np.zeros((nx,nExp),dtype=complex)
omega[:,0]=c*k   # exact
omega[:,1]=c*np.sin(k*dx)/dx+1j*(np.cos(k*dx)-1)   # 1st order
omega[:,2]=c*np.sin(k*dx)/dx   # 2nd order
omega[:,3]=c/dx*(4/3*np.sin(k*dx)-1/6*np.sin(2*k*dx)-1j*(1-np.cos(k*dx))**2/3)
omega[:,4]=c/dx*(4/3*np.sin(k*dx)-1/6*np.sin(2*k*dx))

# allocate numerical solution, initialize with zeros
phi=np.zeros((nt+1,nx,nExp))
phi0=np.zeros(nx);phi0[nx//2]=1  # spike
phi[0,:,:]=np.resize(phi0,(nExp,nx)).T

# evolution in time
for jt in range(1,nt+1):
    for jExp in range(nExp):
        phi_s=np.fft.fft(phi[jt-1,:,jExp])  # spectral components
        phi_s=np.exp(-1j*omega[:,jExp]*dt)*phi_s
        phi[jt,:,jExp]=np.real(np.fft.ifft(phi_s))

### 2nd and 4th order schemes

In [ ]:
# create animation
create_animation(phi=phi[:,:,[0,2,4]],labels=['exact','2nd order','4th order'],ylim=[-.5,1.2])

### 1st and 3rd order schemes

In [ ]:
create_animation(phi=phi[:,:,[0,1,3]],labels=['exact','1st order','3rd order'],ylim=[-.5,1.2])

## Response to smooth initial state

In [ ]:
# parameters
dt=1.0
nt=12
nx=30
dx=2*np.pi/nx
c=dx/dt

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# wavenumbers: conventional order is (0:n/2-1,-n/2:-1)
k=(np.arange(nx)+nx//2)%nx-nx//2

# dispersion relation
nExp=5
omega=np.zeros((nx,nExp),dtype=complex)
omega[:,0]=c*k   # exact
omega[:,1]=c*np.sin(k*dx)/dx+1j*(np.cos(k*dx)-1)   # 1st order
omega[:,2]=c*np.sin(k*dx)/dx   # 2nd order
omega[:,3]=c/dx*(4/3*np.sin(k*dx)-1/6*np.sin(2*k*dx)-1j*(1-np.cos(k*dx))**2/3)
omega[:,4]=c/dx*(4/3*np.sin(k*dx)-1/6*np.sin(2*k*dx))

# allocate numerical solution, initialize with zeros
phi=np.zeros((nt+1,nx,nExp))
phi0=np.cos(4*x)+np.cos(3*x) # smooth function
phi[0,:,:]=np.resize(phi0,(nExp,nx)).T

# evolution in time
for jt in range(1,nt+1):
    for jExp in range(nExp):
        phi_s=np.fft.fft(phi[jt-1,:,jExp])  # spectral components
        phi_s=np.exp(-1j*omega[:,jExp]*dt)*phi_s
        phi[jt,:,jExp]=np.real(np.fft.ifft(phi_s))

# animate results
create_animation(phi,labels=['exact','1st order','2nd order','3rd order','4th order'],ylim=[-3,3])